In [31]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

train_df = pd.read_csv('./train.csv')

def get_images(df):
    df = df.copy()
    labels = df['label'].values
    enc = OneHotEncoder(sparse=False)
    enc.fit(np.array(range(10)).reshape(-1,1))
    labels = enc.transform(labels.reshape(-1,1))
    
    del df['label']
    
    images = df.values.reshape(len(df), 28, 28, 1)
    return labels, images

def view_images(images, labels):
    num_cols = int(np.ceil(np.sqrt(len(images))))
    fig, axs = plt.subplots(nrows=num_cols,ncols=num_cols)
    axs = axs.flatten()

    for idx, ax in enumerate(axs):
        ax.imshow(images[idx].reshape(28,28))
        ax.axes.set_title(np.argmax(labels[idx]))
    plt.show()



In [34]:
import tensorflow as tf
tf.reset_default_graph()

def conv_layer(X, input_channels, output_channels, name):
    with tf.name_scope(name):
        W = tf.Variable(tf.random_normal(
            [3,3,input_channels,output_channels],stddev=.01), name='W1')
        tf.summary.histogram('weights', W)
        l1 = tf.nn.conv2d(X, filter=W, strides=[1,1,1,1], padding='SAME')
        l1 = tf.nn.relu(l1)
        
        return l1

def fc_layer(X, input_channels, output_channels, name):
    with tf.name_scope(name):
        W = tf.Variable(tf.random_normal([input_channels, output_channels]), name='W2')
        l1 = tf.matmul(X, W)
        return l1
    
def train_model():
    labels, images = get_images(train_df)
    images = (images-np.mean(images))/np.std(images)
    trX, teX, trY, teY = train_test_split(images, labels, test_size=.2, random_state=42)
    #img_mn = tf.constant(np.mean(trX), 'float32')
    #img_std = tf.constant(np.std(trX), 'float32')
    
    X = tf.placeholder('float32', [None, 28, 28, 1], name='x')     #28x28x1
    Y = tf.placeholder('float32', [None, 10], name='y')
    
    #X = tf.divide(tf.subtract(X,img_mn), img_std)
    h = conv_layer(X, 1, 32, 'convlayer1')                           
    h = tf.reshape(h, [-1, 28*28*32])
    Yp = fc_layer(h, 28*28*32, 10, 'fc1')
    
    with tf.name_scope('xent'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Yp, labels=Y))
    
    with tf.name_scope('train'):
        train_op = tf.train.GradientDescentOptimizer(.01).minimize(cost)
    
    with tf.name_scope("accuracy"):
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.arg_max(Yp,1),tf.arg_max(Y,1)),tf.float32))
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        merged = tf.summary.merge_all()
        fw = tf.summary.FileWriter('./hope/1', sess.graph, flush_secs=5)
        
        for i in range(50):
            for start, end in zip(range(0,len(trX), 100), range(100, len(trX)+1, 100)):
                
                sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end]})
                
            summary = sess.run(merged, feed_dict={X:teX, Y:teY})
            fw.add_summary(summary, i)
            print sess.run(cost, feed_dict={X:teX, Y:teY})
            print 'Training Acc', sess.run(acc, feed_dict={X:trX, Y:trY})
            print 'Validation: ', sess.run(acc, feed_dict={X:teX, Y:teY})
          
        
            
    

In [35]:
train_model()

9.27318
Training Acc 0.59744
Validation:  0.591905
6.26333
Training Acc 0.649018
Validation:  0.645833
7.17587
Training Acc 0.621042
Validation:  0.615119
6.22235
Training Acc 0.663452
Validation:  0.659167
2.41254
Training Acc 0.755387
Validation:  0.750952
2.34529
Training Acc 0.759464
Validation:  0.756667
2.09091
Training Acc 0.770952
Validation:  0.776667
1.78778
Training Acc 0.803125
Validation:  0.798333
1.65533
Training Acc 0.808214
Validation:  0.802262
1.69463
Training Acc 0.812381
Validation:  0.809048
1.49117
Training Acc 0.813125
Validation:  0.805595
1.48882
Training Acc 0.820952
Validation:  0.822738
1.80503
Training Acc 0.794554
Validation:  0.794643
1.28589
Training Acc 0.833274
Validation:  0.831429
1.25417
Training Acc 0.832649
Validation:  0.832262
1.20063
Training Acc 0.83631
Validation:  0.828214
1.25397
Training Acc 0.836012
Validation:  0.837976
1.20121
Training Acc 0.837202
Validation:  0.829405
1.2527
Training Acc 0.835387
Validation:  0.830357
1.14523
Trainin

KeyboardInterrupt: 